In [1]:
import json
import os
import re
import zipfile
import networkx as nx

In [34]:
# import torch

In [2]:
with open("Q2_2023_Air_Canada_MDA_P19/structuredData.json", 'r') as json_file:
    # Load the JSON data
    p19 = json.load(json_file)

In [3]:
with open("Q2_2023_Air_Canada_MDA_P20/structuredData.json", 'r') as json_file:
    # Load the JSON data
    p20 = json.load(json_file)

In [4]:
p19.keys()

dict_keys(['version', 'extended_metadata', 'elements', 'pages'])

In [5]:
# for element in p19['elements']:
#     tag_found = False
#     for tag in ['Figure', 'Title', 'H1', 'H2', 'Table']:
#         if tag in element['Path']:
#             tag_found = True
#             break  # Exit the inner loop once a match is found

#     if not tag_found:
#         print(element['Text'], "\n")

In [6]:
all_files = os.listdir("js_output")
all_files.remove(".DS_Store")
all_files.sort(key=lambda f: int(re.sub(r'\D', '', f)))

In [7]:
Q2_files = [file for file in all_files if "Q2" in file]
Q3_files = [file for file in all_files if "Q3" in file]

In [8]:
len(Q2_files), len(Q3_files)

(33, 34)

In [9]:
Q2_files[0]

'Q2_2023_Air_Canada_MDA_P1.zip'

In [10]:
# for element in p19["elements"]:
#     if(element["Path"].endswith("/P")):
#         print(element["Text"])

In [11]:
output_folder = "js_output"
file = "Q2_2023_Air_Canada_MDA"

In [12]:
filtered_elements = []

for file in Q2_files[2:]:
    
    page_no = int(file.split(".")[0].split("_")[-1][1:])
    file_path = os.path.join("js_output", file)
    archive = zipfile.ZipFile(file_path, 'r')
    jsonentry = archive.open('structuredData.json')
    jsondata = jsonentry.read()
    data = json.loads(jsondata)
    
    for element in data['elements']:
        tag_found = False
        for tag in ['Figure', 'Table', 'Reference', 'Lbl']: # exclude Figure, Tabel cells, Reference, List Item Label
            if (tag in element['Path']):
                tag_found = True
                if element['Path'].endswith("/Table"):
                    element['Page'] = page_no
                    filtered_elements.append(element) # collect table element
                break  # Exit the inner loop once a match is found
        
        # Text needs to have at least 5 words unless it's a title
        if (not tag_found) and ((len(element['Text'].split()) >= 5) or ("Title" in element['Path']) or ("H1" in element['Path']) or ("H2" in element['Path'])):
            # print(element['Path'], element['Text'], "\n")
            element['Page'] = page_no
            filtered_elements.append(element) # collect text element

In [13]:
len(filtered_elements)

298

In [14]:
# remove unwanted text
for element in filtered_elements:
    try:
        if element['Text'] == "Second Quarter 2023 Management’s Discussion and Analysis of Results of Operations and Financial Condition ":
            filtered_elements.remove(element)
    except:
        pass
print(len(filtered_elements))

267


In [15]:
section = "Anything"
sections = []

for element in filtered_elements:
    if (element['Path'].endswith("Title")) or (element['Path'].endswith("H1")) or (element['Path'].endswith("H2")):
        # assign section name for all text elements
        section = element['Text']
        sections.append(section)
    element['Section'] = section

In [16]:
len(sections)

66

In [17]:
file

'Q2_2023_Air_Canada_MDA_P33.zip'

In [18]:
for section in sections[:1]:
    print(section)
    section_text = ""
    for element in filtered_elements:
        # concatenate all text in the section and then split into 100-words passages
        if (element['Section'] == section) and not (element['Path'].endswith('/Table')):
            if element['Text'] != section:
                section_text += element['Text'] + " "

1. SELECTED FINANCIAL METRICS AND STATISTICS 


In [19]:
section_text

'The financial and operating highlights for Air Canada for the periods indicated are as follows:  (1) Operating margin is a supplementary financial measure and is defined as operating income (loss) as a percentage of operating revenues.  (2) Adjusted EBITDA (earnings before interest, taxes, depreciation and amortization), adjusted EBITDA margin, adjusted pre-tax income (loss), adjusted net income (loss), free cash flow, net debt, adjusted earnings (loss) per share and adjusted CASM are non-GAAP financial measures, capital management measures, non-GAAP ratios or supplementary financial measures. Such measures are not recognized measures for financial statement presentation under GAAP, do not have standardized meanings, may not be comparable to similar measures presented by other entities and should not be considered a substitute for or superior to GAAP results. Refer to section (<>)16 “Non-GAAP Financial Measures” of this MD&A for descriptions of Air Canada’s non-GAAP financial measures

In [20]:
def split_into_passages(text):
    
    words = text.split()
    num_passages = (len(words) + 99) // 100
    passages = []

    # Iterate through the number of passages
    for i in range(num_passages):
        start_index = i * 100
        end_index = (i + 1) * 100
        passage = ' '.join(words[start_index:end_index])
        passages.append(passage)

    return passages

In [21]:
split_into_passages(section_text)

['The financial and operating highlights for Air Canada for the periods indicated are as follows: (1) Operating margin is a supplementary financial measure and is defined as operating income (loss) as a percentage of operating revenues. (2) Adjusted EBITDA (earnings before interest, taxes, depreciation and amortization), adjusted EBITDA margin, adjusted pre-tax income (loss), adjusted net income (loss), free cash flow, net debt, adjusted earnings (loss) per share and adjusted CASM are non-GAAP financial measures, capital management measures, non-GAAP ratios or supplementary financial measures. Such measures are not recognized measures for financial statement presentation under GAAP, do not have standardized meanings,',
 'may not be comparable to similar measures presented by other entities and should not be considered a substitute for or superior to GAAP results. Refer to section (<>)16 “Non-GAAP Financial Measures” of this MD&A for descriptions of Air Canada’s non-GAAP financial measu

In [22]:
filtered_elements_copy = filtered_elements.copy()

In [23]:
len(filtered_elements_copy), len(filtered_elements)

(267, 267)

In [24]:
updated_elements = []
element_dict = dict()

for section in sections:
    # print(section)
    section_text = ""
    section_tables = []
    for element in filtered_elements_copy:
        # print(element['Section'], element['Path'])
        # concatenate all text in the section and then split into 100-words passages
        if element['Section'] == section:
            if element['Path'].endswith('/Table'):
                # print(element['Path'])
                section_tables.append(element)
            #     filtered_elements_copy.remove(element)
                
            else:
                if element['Text'] != section:
                    # print(element['Text'])
                    section_text += element['Text'] + " "
                # filtered_elements_copy.remove(element)
        else:
            continue
            
        
    section_passages = split_into_passages(section_text)
    for i in range(len(section_passages)):
        element_dict = {"Section": section,
                        "Type": "Passage",
                        "Name": "Passage_" + str(i+1),
                        "Feature": section_passages[i]
                       }
        updated_elements.append(element_dict)
    
    for i, element in enumerate(section_tables):
        element_dict = {"Section": element['Section'],
                "Page": element['Page'],
                "Type": "Table",
                "Name": "Table_" + str(i+1),
                "NumCol": element['attributes']['NumCol'],
                "NumRow": element['attributes']['NumRow'],
                "XlsxPath": os.path.join(output_folder, file+"_P"+str(element['Page']), element['filePaths'][0]),
                "PngPath": os.path.join(output_folder, file+"_P"+str(element['Page']), element['filePaths'][1])
           }
        updated_elements.append(element_dict)

In [25]:
len(updated_elements)

146

In [26]:
updated_elements[:5]

[{'Section': '1. SELECTED FINANCIAL METRICS AND STATISTICS ',
  'Type': 'Passage',
  'Name': 'Passage_1',
  'Feature': 'The financial and operating highlights for Air Canada for the periods indicated are as follows: (1) Operating margin is a supplementary financial measure and is defined as operating income (loss) as a percentage of operating revenues. (2) Adjusted EBITDA (earnings before interest, taxes, depreciation and amortization), adjusted EBITDA margin, adjusted pre-tax income (loss), adjusted net income (loss), free cash flow, net debt, adjusted earnings (loss) per share and adjusted CASM are non-GAAP financial measures, capital management measures, non-GAAP ratios or supplementary financial measures. Such measures are not recognized measures for financial statement presentation under GAAP, do not have standardized meanings,'},
 {'Section': '1. SELECTED FINANCIAL METRICS AND STATISTICS ',
  'Type': 'Passage',
  'Name': 'Passage_2',
  'Feature': 'may not be comparable to similar

In [27]:
def create_knowledge_graph(data):
    # Create a directed graph
    knowledge_graph = nx.DiGraph()

    # Iterate through the list of dictionaries
    for item in data:
        section_name = item["Section"]

        # Add section node if not already added
        section_node = {'Type': "Section",
                        "Name": section_name
                       }
        knowledge_graph.add_nodes_from([(section_name, section_node)])

        if item["Type"] == "Table":
            # Add table node
            table_name = ", ".join([section_name, item['Name']])
            knowledge_graph.add_nodes_from([(table_name, item)])

            # Add directed edge from section to table
            knowledge_graph.add_edge(section_name, table_name)

        elif item["Type"] == "Passage":
            # Add passage node
            passage_name = ", ".join([section_name, item['Name']])
            knowledge_graph.add_nodes_from([(passage_name, item)])

            # Add directed edge from section to passage
            knowledge_graph.add_edge(section_name, passage_name)

    return knowledge_graph


In [28]:
# Create the knowledge graph
graph = create_knowledge_graph(updated_elements)
len(list(graph.nodes))

204

In [32]:
len(list(graph.edges))

142

In [31]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

NameError: name '_C' is not defined

In [ ]:
msmarco-distilbert-base-tas-b